# QSVM on the Iris Dataset

### Step 1: Load in the Quantum Libaries

In [ ]:
ionq_token=""

In [ ]:
from qiskit import QuantumCircuit, execute, Aer
from qiskit.circuit.library import ZFeatureMap
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit_machine_learning.algorithms import QSVC
from qiskit_machine_learning.kernels import QuantumKernel

from qiskit_ionq import IonQProvider

provider = IonQProvider(ionq_token)
# show supported backends
print(provider.backends()) 

# create backends for 'ionq_simulator' and 'ionq_qpu'
ionq_sim = provider.get_backend("ionq_simulator")
ionq_qpu = provider.get_backend("ionq_qpu")

### Step 2: Load in Maths and Plotting Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import svm, datasets, model_selection, decomposition
import seaborn as sns
sns.set_style("whitegrid")
sns.set_palette("pastel")

%matplotlib inline

### Step 3: Load in the Iris dataset

Here we are restricting to only the first 2 classes, a multi class classification can be extrapolated.

In [ ]:
iris = datasets.load_iris()
X_raw = iris.data[:100] # Use features 1 and 2 as our X values
y_raw = iris.target[:100]
X, X_test, y, y_test = model_selection.train_test_split(X_raw, 
                                y_raw, test_size=0.33, random_state=42)

### Plotting Helper

This plots both the data and the classification boundaries

In [ ]:
def draw_plot(data, labels, model=None, pca=None):
    fig1, ax1 = plt.subplots()
    ax1.set_xlabel("sepal width (cm)")
    ax1.set_ylabel("petal length (cm)")
    
    if pca is not None:
        data = pca.transform(data)
        
    df = pd.DataFrame(data)
    df["target"] = labels

    for name, group in df.groupby('target'):
        ax1.scatter(x=group[0], y=group[1], label=iris['target_names'][name])

    if model is not None:
        xlim, ylim = ax1.get_xlim(), ax1.get_ylim()

        # create grid to evaluate model
        y_grid, x_grid = np.meshgrid(np.linspace(ylim[0], ylim[1], 30), np.linspace(xlim[0], xlim[1], 30))
        xy = np.vstack([x_grid.ravel(), y_grid.ravel()]).T
        if pca is not None:
            P = model.decision_function(pca.inverse_transform(xy)).reshape(x_grid.shape)
        else:
            P = model.decision_function(xy).reshape(x_grid.shape)


        margin_color, decision_color = sns.color_palette()[4], sns.color_palette()[3]
        # plot decision boundary and margins
        ax1.contour(x_grid, y_grid, P, 
                    colors=[margin_color, decision_color, margin_color],
                    levels=[-1, 0, 1], alpha=0.5,
                    linestyles=['--', '-', '--'])
    
    if pca is not None:
        plt.title("PCA Reduced Features of Setosa and Versicolor Flowers")
    else:
        plt.title("First Two Features of Setosa and Versicolor Flowers")
    ax1.legend(loc="upper right")

### Step 4: Observe the first two dimensions of the data

In [ ]:
draw_plot(X,y)

### Step 5: Fit the classical solver

Using just two dimensions here

In [ ]:
classical = svm.SVC(kernel='linear')
classical.fit(X[:, 1:3], y)

### Step 6: Visualize the Classification boundaries

In [ ]:
draw_plot(X[:,1:3],y, model=classical)

### Step 7: Initialize the QSVC Solver

In [ ]:
# number of qubits is equal to the number of features
num_qubits = 2

# number of steps performed during the training procedure
tau = 100

# regularization parameter
C = 1000

feature_map = ZFeatureMap(feature_dimension=num_qubits, reps=3)
qkernel = QuantumKernel(feature_map=feature_map, quantum_instance=Aer.get_backend("aer_simulator"))

### Step 8: Fit the QSVC Solver

In [ ]:
qsvc = QSVC(quantum_kernel=qkernel)
qsvc.fit(X,y)

In [ ]:
correct = sum(qsvc.predict(X_test)== y_test).astype(int)
total = len(y_test)
print(f"Classifier Accuracy on Test Set: {(correct/total)*100} %")

### Step 9: Visualizing the QSVC Decision boundaries

# The QSVM needs 2+ feature dims

Use PCA to visualize:

In [ ]:
pca = decomposition.PCA(n_components=2)
pca.fit(X)

In [ ]:
draw_plot(X, y, model=qsvc, pca=pca)

# Apply the QSVM to your own dataset

Thoughts:
1. Consider switching to a statevector simulator for improved speed
2. Consider an optimal dimensionality for the QKernel
3. Attempt alternative Qkernels